In [2]:
# Lab 6 Sotfmax zoo classifier
import torch
from torch.autograd import Variable
import numpy as np
import sys

# Set random Seed
torch.manual_seed(111)

# 1. Data loading
xy=np.loadtxt('../00_data/data-04-zoo.csv',delimiter=',',dtype=np.float32)
x_data=xy[:,:-1]
y_data=xy[:,[-1]]

print(x_data.shape,y_data.shape)
nb_classes=7

# ndarray를 torch variable 로 변형
X=Variable(torch.from_numpy(x_data))
Y=Variable(torch.from_numpy(y_data))

# 2. one_hot encoding : 이진분류 문제이므로,0~6 사이의 정수값을 갖는 Y를 one-hot encoding 하여 이진분류

Y_one_hot=torch.zeros(Y.size()[0],nb_classes) # 101,7 크기의 torch list를 0으로 채운 torch list return
Y_one_hot.scatter_(1,Y.long().data,1) # self.long()`` is equivalent to ``self.to(torch.int64)``.
Y_one_hot=Variable(Y_one_hot)
print("one_hot",Y_one_hot.data)

# 3. Model instance 생성
softmax=torch.nn.Softmax()
model=torch.nn.Linear(16,nb_classes,bias=True)

# Cross entropy cost/loss
criterion=torch.nn.CrossEntropyLoss() # Softmax is internally computed
optimizer=torch.optim.SGD(model.parameters(),lr=0.1)

# 4. Model Train

for step in range(2001):
    optimizer.zero_grad()
    hypothesis=model(X)
    # Label has to be 1D Long Tensor
    # : Y.long()을 사용하여 64bit long int type으로 변환.
    cost=criterion(hypothesis,Y.long().view(-1)) 
    # view는 차원 재구성. reshape()과 같이 차원을 재구서하는 매소드. Y.long() 의 배열은 1차원배열로 출력.
    # Y.size() ==> (101,1) Y.view(-1).size() ==> (101) 
    # (EX) 
    # Y.view(1,2,3) Y.reshape(1,2,3) 처럼 변경하고싶은 shape을 tuple로 입렵. 
    # view는 기존의 데이터와 같은 메모리 공간을 공유.
    # view(-1) 은 reshape(-1) 과 같이 1차원의 배열을 반환함.
    
    cost.backward()
    optimizer.step()
    

# 5. Ptrdiction
    prediction=torch.max(softmax(hypothesis),1)[1].float()
    '''
    torch.max 함수 
    - 주어진 tensor 배열의 최댓값이 들어있는 index를 리턴하는 함수
    - Y_pred=[[0.3,0.2,0.9,0.1]] 의 경우 torch.max(Y_pred.data,1) 의 결과는 0.9의 인덱스인 2가 된다.
    - 뒤에 들어가는 1은 dimmension에 대한 것.
    '''    
    correct_prediction=(prediction.data==Y.data) # 예측값인 prediction 와 실측값인 Y가 같으면 1을 반환
    accuracy=correct_prediction.float().mean() # Prediction의 평균을 구하여 정확도를 반환.

    if step%100==0:
        print("Step:{:5}\tLoss:{:.3f}\tAcc:{:.2%}".format(step,cost.data,accuracy))
# Let's see if we can preidct
pred=torch.max(softmax(hypothesis),1)[1].float()
for p,y in zip(pred,Y):
    print("[{}]Prediction:{} True Y:{}".format(bool(p.data==y.data),p.data,y_data[0]))



(101, 16) (101, 1)
one_hot tensor([[1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0.

https://kjhov195.github.io/2020-01-04-softmax_classifier_2/</br>

Cross Entropy : https://medium.com/data-science-bootcamp/understand-cross-entropy-loss-in-minutes-9fb263caee9a

</br> 
https://nuguziii.github.io/paper-review/PR-001/


In [7]:
# view매서드의 이해를 위한 test
print(Y.size())
#print(Y)
print(Y.view(-1).size())
print(Y.view(-1))
print("-------------------------------")
print(Y.long())


# max 매서드의 이해를 위한 test 
test_np=np.array(range(30)).reshape(6,5)
test_tr=Variable(torch.from_numpy(test_np))

print(test_tr.data)
print(torch.max(test_tr,0)) # 0은 2차원 배열의 각 행의 최댓값의 값과 index를 반환해준다.
print(torch.max(test_tr,1)) # 1은 2차원 배열의 각 열의 최댓값의 값과 index를 반환해 준다.
print('-----------------------------------------')
print(torch.max(test_tr,0)[0]) # [0] 은 값과 dtype을
print(torch.max(test_tr,0)[1]) # [1] 은 index를 반환한다.




torch.Size([101, 1])
torch.Size([101])
tensor([0., 0., 3., 0., 0., 0., 0., 3., 3., 0., 0., 1., 3., 6., 6., 6., 1., 0.,
        3., 0., 1., 1., 0., 1., 5., 4., 4., 0., 0., 0., 5., 0., 0., 1., 3., 0.,
        0., 1., 3., 5., 5., 1., 5., 1., 0., 0., 6., 0., 0., 0., 0., 5., 4., 6.,
        0., 0., 1., 1., 1., 1., 3., 3., 2., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        6., 3., 0., 0., 2., 6., 1., 1., 2., 6., 3., 1., 0., 6., 3., 1., 5., 4.,
        2., 2., 3., 0., 0., 1., 0., 5., 0., 6., 1.])
-------------------------------
tensor([[0],
        [0],
        [3],
        [0],
        [0],
        [0],
        [0],
        [3],
        [3],
        [0],
        [0],
        [1],
        [3],
        [6],
        [6],
        [6],
        [1],
        [0],
        [3],
        [0],
        [1],
        [1],
        [0],
        [1],
        [5],
        [4],
        [4],
        [0],
        [0],
        [0],
        [5],
        [0],
        [0],
        [1],
        [3],
        [0],
        